### 1 - Import test data

In [6]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-07-10", end="2024-07-18", interval='15m')
dataF.iloc[:,:]
dataF.Open.iloc

dataF.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-10 00:00:00,1.081783,1.081900,1.081783,1.081900,1.081900,0
2024-07-10 00:15:00,1.081900,1.081900,1.081783,1.081900,1.081900,0
2024-07-10 00:30:00,1.081900,1.081900,1.081783,1.081783,1.081783,0
2024-07-10 00:45:00,1.081783,1.081783,1.081783,1.081783,1.081783,0
2024-07-10 01:00:00,1.081783,1.081900,1.081666,1.081666,1.081666,0


### 2 - Define your signal function

In [7]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
# signal_generator(data)
dataF["signal"] = signal

In [8]:
dataF.signal.value_counts()
print(dataF.signal.value_counts())
dataF.iloc[:, :]

signal
0    546
2     14
1     10
Name: count, dtype: int64


,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2024-07-10 00:00:00,1.081783,1.081900,1.081783,1.081900,1.081900,0,0
2024-07-10 00:15:00,1.081900,1.081900,1.081783,1.081900,1.081900,0,0
2024-07-10 00:30:00,1.081900,1.081900,1.081783,1.081783,1.081783,0,0
2024-07-10 00:45:00,1.081783,1.081783,1.081783,1.081783,1.081783,0,0
2024-07-10 01:00:00,1.081783,1.081900,1.081666,1.081666,1.081666,0,0
...,...,...,...,...,...,...,...
2024-07-17 22:45:00,1.094212,1.094331,1.094092,1.094212,1.094212,0,0
2024-07-17 23:00:00,1.094212,1.094212,1.094212,1.094212,1.094212,0,0
2024-07-17 23:15:00,1.094212,1.094212,1.094212,1.094212,1.094212,0,0


### 3 - Connect to the market and execute trades

In [10]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [16]:
import configparser
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    config = configparser.ConfigParser()
    config.read('config.cfg')
    access_token = config.get( 'oanda','access_token')
    #
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)

for candle in candles:
    print(float(str(candle.bid.o))>1)


MissingSectionHeaderError: File contains no section headers.
file: 'config.cfg', line: 2
'access_token = 9da2fe1d4f864bc71e017e1214250d11-a9a419d2549cf5dc1a364f15e58b62cf'

In [ ]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()